In [3]:
import mysql.connector as sql
import pandas as pd
import datetime as dt
import numpy as np
from sorted_months_weekdays import *
from sort_dataframeby_monthorweek import *
import matplotlib.pyplot as plt
from scipy.stats.mstats import mode
import json
import pickle
from itertools import chain, combinations

In [6]:
  f = open("/Users/shirin/other_projects/cooper_hewitt/Pen_Data_Dictionary/relation_counts.json", "rb")
  data = pickle.load(f)
  f.close()

In [13]:
pd.DataFrame.from_dict(data,orient='index')

,0
"(68268461, 18802401, 68268251)",2
"(35457321, 18325257, 68268241, 68268227)",2
"(35457109, 51866827)",2
"(35520549, 18802401, 68268243, 68268229)",2
"(35457321, 35460389, 35520549, 68268235)",2
...,...
"(51689393, 18325257, 35457109, 18802401)",29
"(35457317, 35520541, 35460389)",29
"(35457317, 18325257, 35460389, 18802401)",29
"(51689393, 35457319, 18325257, 18802401)",29


In [3]:
db_connection = sql.connect(host='localhost', database='collection_visits', user='pandas', password='password')

visitsItems = pd.read_sql('SELECT * FROM VisitsItems', con=db_connection)
visitsItems['created'] = pd.to_datetime(visitsItems['created'], unit='s')
visitsItems['lastmodified'] = pd.to_datetime(visitsItems['lastmodified'], unit='s')
visitsItems['deleted'].replace(0, np.nan, inplace=True)
visitsItems['deleted'] = pd.to_datetime(visitsItems['deleted'], unit='s')
visitsItems['title'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
visitsItems['description'].replace(r'^\s*$', np.NaN, regex=True, inplace=True)
visitsItems['title'] = visitsItems['title'].astype('category')
visitsItems['description'] = visitsItems['description'].astype('category')

In [4]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 50)
pd.set_option("max_colwidth", 500)

In [5]:
#in order to look by visits id.
#i can look by created - by day, items that usually were collected together by day. 
#or ii can look by user id - but wwith user 0. 

In [6]:
#starting without user == 0. to merge on user id
visitsItems_clean = visitsItems.drop(['refers_to_id', 'deleted','status_id', 'processed', 'input_id'], axis=1)
visitsItems_clean = visitsItems_clean[visitsItems_clean['user_id'] !=0]

In [7]:
visitsItems_small = visitsItems[['user_id', 'refers_to_uid', 'created']]

In [8]:
visitsItems_small = visitsItems_small[visitsItems_small['user_id'] != 0]

In [9]:
visitsItems_small

,user_id,refers_to_uid,created
3,3,18476565,2014-10-23 16:27:59
4,3,18628929,2014-10-23 19:36:56
5,3,18466243,2014-10-23 19:53:54
6,3,18445757,2014-10-23 19:56:42
7,3,18634081,2014-10-24 02:05:21
...,...,...,...
18479552,28814,1595448288,2020-07-22 20:04:48
18479554,28814,1595449610,2020-07-22 20:26:50
18479555,28814,1595449666,2020-07-22 20:27:46
18479556,28814,1595450275,2020-07-22 20:37:55


In [38]:
f = lambda x: mode(x, axis=None)[0]
visitsItems_small.groupby('user_id').apply(f)

user_id
3            [3.0]
142        [142.0]
158        [158.0]
247        [247.0]
283        [283.0]
           ...    
30271    [30271.0]
30383    [30383.0]
30401    [30401.0]
30414    [30414.0]
30486    [30486.0]
Length: 1468, dtype: object

In [9]:
visitsItems_small.set_index('created', inplace=True)

In [10]:
visitsItems_opening_hours = visitsItems_small.between_time('10:00', '18:00')

In [11]:
visitsItems_opening_hours.reset_index(inplace=True)

In [13]:
visitsItems_opening_group = visitsItems_opening_hours.groupby(['user_id', 'refers_to_uid'])
visitsItems_opening_group.first().head(500)

created
user_id refers_to_uid                    
3       18220487      2015-02-02 17:02:55
        18325257      2015-02-20 17:49:21
        18394375      2014-10-24 15:16:14
        18476565      2014-10-23 16:27:59
        18489807      2014-10-24 16:08:46
        18500009      2014-10-27 17:59:39
        18503039      2015-02-24 17:15:56
        18620817      2014-12-16 17:15:55
        18639109      2015-02-20 17:48:38
        18648865      2014-12-16 17:47:49
        18649321      2015-02-20 17:45:51
        18703483      2015-02-20 17:47:26
        18705931      2015-02-20 17:45:33
        18710413      2014-12-16 17:48:58
        18730767      2015-02-24 17:16:00
        18802401      2015-02-20 17:49:21
        18805891      2015-02-24 17:16:16
        35456853      2015-02-20 17:45:33
        35457109      2015-02-20 17:49:21
        35457285      2015-02-20 17:48:45
        35457317      2015-02-20 17:49:21
        35457319      2015-02-20 17:49:21
        35457321      2015-02-20 17:49:21
        35460389      2015-02-20 17:49:21
        35460703      2015-02-20 17:49:05
        35460713      2015-02-20 17:49:21
        35460835      2015-02-20 17:45:33
        35520541      2015-02-20 17:49:21
        35520549      2015-02-20 17:49:21
        35520989      2015-02-20 17:48:20
        51497635      2015-02-20 17:45:59
        51689393      2015-02-20 17:49:21
        51866827      2015-02-20 17:46:13
        68244063      2015-02-20 17:45:33
        68245549      2015-02-20 17:45:33
        68268059      2015-02-20 17:45:33
        68268227      2015-02-20 17:46:36
        68268229      2015-02-20 17:46:36
        68268233      2015-02-20 17:46:36
        68268235      2015-02-20 17:46:36
        68268237      2015-02-20 17:46:36
        68268239      2015-02-20 17:46:36
        68268241      2015-02-20 17:46:36
        68268243      2015-02-20 17:46:36
        68268245      2015-02-20 17:46:36
        68268247      2015-02-20 17:46:36
        68268251      2015-02-20 17:46:36
        68268253      2015-02-20 17:46:36
        68268255      2015-02-20 17:46:36
        68268461      2015-02-20 17:49:21
        68731001      2015-02-20 17:45:33
        68731009      2015-02-20 17:45:33
        68743599      2015-02-20 17:46:40
142     18102519      2015-03-05 13:44:05
        18105927      2015-03-05 13:41:39
        18112399      2015-03-05 13:42:05
        18122837      2015-03-05 13:42:07
        18133903      2015-03-05 13:41:44
        18134343      2015-03-05 13:41:42
        18180127      2015-03-05 13:41:54
        18187735      2015-03-05 13:41:34
        18312379      2015-03-05 13:41:48
        18312733      2015-03-05 13:41:57
        18312735      2015-03-05 13:41:57
        18316217      2015-03-05 13:41:32
        18318573      2015-03-05 13:41:52
        18348701      2015-03-05 13:42:50
        18349723      2015-03-05 13:42:03
        18407533      2015-03-05 13:46:34
        18429505      2015-03-05 13:52:54
        18444031      2015-03-05 13:44:28
        18468717      2015-03-05 13:42:48
        18471589      2015-03-05 13:45:08
        18490057      2015-03-05 13:45:41
        18500703      2015-03-05 13:44:34
        18610125      2015-03-05 13:44:59
        18633615      2015-03-05 13:58:34
        18634859      2015-03-05 13:44:29
        18636425      2015-03-05 13:40:00
        18637815      2015-03-05 13:47:39
        18643095      2015-03-05 13:42:45
        18649487      2015-03-05 13:46:33
        18697983      2015-03-05 13:47:35
        18703479      2015-03-05 13:38:46
        18703485      2015-03-05 13:38:48
        18705481      2015-03-05 13:42:42
        18705675      2015-07-02 14:30:49
        18705867      2015-03-05 13:45:06
        18709775      2015-03-05 13:49:08
        18712511      2015-03-05 13:45:36
        18729815      2015-03-05 13:43:31
        18730781      2015-03-05 13:47:31
        18731999      2015-03-05 13:44:31
        18732295      2015-03-05 13:58:32
  

In [81]:
visitsItems_opening_group_first = visitsItems_opening_group.first()

In [101]:
visitsItems_opening_group_first

created
user_id refers_to_uid                    
3       18220487      2015-02-02 17:02:55
        18325257      2015-02-20 17:49:21
        18394375      2014-10-24 15:16:14
        18476565      2014-10-23 16:27:59
        18489807      2014-10-24 16:08:46
...                                   ...
30401   2318800187    2020-02-16 17:04:29
        2318800201    2020-02-16 17:06:19
        2318800227    2020-02-16 17:08:51
        2318800228    2020-02-16 17:08:17
        2318800232    2020-02-16 17:12:44

[34274 rows x 1 columns]

In [15]:
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [16]:
list(powerset(visitsItems_list[2]))

[(),
 (18155285,),
 (18188419,),
 (18349933,),
 (18633053,),
 (18648931,),
 (18752821,),
 (18328673,),
 (18430127,),
 (18155285, 18188419),
 (18155285, 18349933),
 (18155285, 18633053),
 (18155285, 18648931),
 (18155285, 18752821),
 (18155285, 18328673),
 (18155285, 18430127),
 (18188419, 18349933),
 (18188419, 18633053),
 (18188419, 18648931),
 (18188419, 18752821),
 (18188419, 18328673),
 (18188419, 18430127),
 (18349933, 18633053),
 (18349933, 18648931),
 (18349933, 18752821),
 (18349933, 18328673),
 (18349933, 18430127),
 (18633053, 18648931),
 (18633053, 18752821),
 (18633053, 18328673),
 (18633053, 18430127),
 (18648931, 18752821),
 (18648931, 18328673),
 (18648931, 18430127),
 (18752821, 18328673),
 (18752821, 18430127),
 (18328673, 18430127),
 (18155285, 18188419, 18349933),
 (18155285, 18188419, 18633053),
 (18155285, 18188419, 18648931),
 (18155285, 18188419, 18752821),
 (18155285, 18188419, 18328673),
 (18155285, 18188419, 18430127),
 (18155285, 18349933, 18633053),
 (181552

In [23]:
visitsItems_list = visitsItems_opening_hours.groupby('user_id')['refers_to_uid'].apply(pd.Series.tolist).tolist()
file = json.dumps(visitsItems_list)

In [24]:
text_file = open("listoflist.txt", "w")
text_file.write(file)
text_file.close()